In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install findspark
!pip install pyspark
!pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 76.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=4bbaf0b12a1ee309b98dbb34c1fa0f116ce480000debec60c6b110f775e3d90e
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 648 kB 4.7 MB/s 


In [7]:
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from sparknlp.annotator import *
from sparknlp.base import *


In [8]:
spark = sparknlp.start(gpu=True)
sc = spark.sparkContext

In [9]:
submissions_en = spark.read.parquet("/content/drive/Shareddrives/502_project/data/submissions_en_cleaned")

In [10]:
twitter_pipeline = PretrainedPipeline("analyze_sentimentdl_use_twitter", lang = "en") 

analyze_sentimentdl_use_twitter download started this may take some time.
Approx size to download 935.1 MB
[OK!]


In [11]:
submissions_en = submissions_en.withColumn('text', col('cleaned_text'))

In [12]:
submissions_en = submissions_en.withColumn('timestamp', col('created_utc')\
                                           .cast('TimeStamp')).withColumn('day', F.date_format(col('timestamp'), 'yyyy-MM-dd'))


In [19]:
submissions_en = twitter_pipeline.transform(submissions_en)
submissions_en = submissions_en.withColumn('sent_score', F.when(col('sentiment').getItem(0).result=='positive', 1.0).otherwise(0.0) )
submissions_en = submissions_en.withColumn('sent', col('sentiment').getItem(0).result )
submissions_en = submissions_en.drop('document','sentence_embeddings','sentiment')
submissions_en.write.mode('overwrite').format("parquet").save("/content/drive/Shareddrives/502_project/data/submissions_en_sent")

In [14]:
### average sentiment
submissions_en.select('sent_score','day').groupBy('day')\
.agg(F.avg('sent_score').alias('avg_sentiment'))\
.sort('day').toPandas().\
to_csv('/content/drive/Shareddrives/502_project/data/RU_submission_en_sentiment/avg_sent_twit.csv',index=False)

In [15]:
### value counts of sentiment
submissions_en.select('sent','day')\
.groupBy('day','sent').agg(F.count('sent').alias('count')).sort('day').toPandas().\
to_csv('/content/drive/Shareddrives/502_project/data/RU_submission_en_sentiment/count_sent_twit.csv',index=False)